# Exemplos `RSelenium`

In [ ]:
# renv é um gerenciador de ambientes para R
renv::init()
renv::install()

Selecionando a porta de rede para abrir o Selenium Server

In [ ]:
port <- 4445L

Iniciando o Selenium Server e o Remote Driver

In [ ]:
# Inicia Selenium Server na porta escolhida
geckodriver <- wdman::selenium(port = port, verbose = FALSE)

# Inicia o Remote Driver
rem_dr <- RSelenium::remoteDriver(
  remoteServerAddr = "localhost",
  port = port,
  browserName = "firefox"
)

# Código para inciar o Remote Driver de forma 'headless'
# options <- list("moz:firefoxOptions" = list(args = list("--headless")))
# rem_dr <- RSelenium::remoteDriver(
#   remoteServerAddr = "localhost",
#   port = port,
#   browserName = "firefox",
#   extraCapabilities = options
# )

Abrindo uma janela do navegador

In [ ]:
rem_dr$open()

Maximizar a tela

In [ ]:
rem_dr$maxWindowSize()

## Exemplos de extracão de filmes com busca (https://www.imdb.com/)

Navegar até o site do IMDB

In [ ]:
rem_dr$navigate("https://www.imdb.com/")

Criar conexão de arquivo para salvar os dados

In [ ]:
file_conn <- file("../../data/rselenium/filmes.txt", open = "a")

In [ ]:
series <- c("WANDINHA","A CASA DO DRAGÃO", "THE BOYS")

header <- c("title", "score", "raiting", "popularity")
writeLines(paste(header, collapse = ";"), file_conn, sep = "\n")

for(serie in series) {
  search <- rem_dr$findElement(using = "xpath","//input[@id=\"suggestion-search\"]")
  search$sendKeysToElement(list(serie, key = "enter"))
  Sys.sleep(2)

  result <- rem_dr$findElement(using = "xpath", "//*[@id=\"__next\"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div[1]/a")
  result$clickElement()
  Sys.sleep(2)

  rating <- rem_dr$findElement(using = "css selector", ".sc-bde20123-1")
  rating <- rating$getElementText()

  score <- rem_dr$findElement(using = "css selector", ".sc-bde20123-3")
  score <- avaliacoes$getElementText()

  popularity <- rem_dr$findElement(using = "css selector", ".sc-5f7fb5b4-1")
  popularity <- popularity$getElementText()

  line <- c(serie, rating, score, popularity)
  writeLines(paste(line, collapse = ";"), file_conn, sep = "\n")

  rem_dr$goBack()
  rem_dr$goBack()
  Sys.sleep(2)
}

Fecha conexão com o arquivo

In [ ]:
close(file_conn)

Finaliza conexão do Remote Driver e Selenium Server

In [ ]:
rem_dr$close()
geckodriver$stop()

## Exemplos de extracão de informações no Airbnb (https://airbnb.com.br/)

In [ ]:
port <- 4445L

geckodriver <- wdman::selenium(port = port, verbose = FALSE)

rem_dr <- RSelenium::remoteDriver(
  remoteServerAddr = "localhost",
  port = port,
  browserName = "firefox"
)

In [ ]:
rem_dr$open()
rem_dr$navigate("https://www.airbnb.com.br")

In [ ]:
rem_dr$findElement(using="css selector",'button.ffgcxut:nth-child(1) > div')$clickElement()
local = rem_dr$findElement('css selector','#bigsearch-query-location-input')
local$sendKeysToElement(list("rio de janeiro", key = "enter"))
buscar = rem_dr$findElement(using="css selector",'html.scrollbar-gutter.js-focus-visible.g5l85gq body.with-new-header div div div.t1bgcr6e div div div.cd56ld.cb80sj1.dir.dir-ltr div.h1ta6hky.dir.dir-ltr div.c1yo0219.dir.dir-ltr div div.dir.dir-ltr header.curo97c.c1tbb4m8.dir.dir-ltr div.c1s03i8x.c17xuogm.dir.dir-ltr div.c1ujpdn9.dir.dir-ltr div.bdypfvo.bfxfvmm.b192n4t3.b13hn6dm.dir.dir-ltr div div.ij4qtlz.dir.dir-ltr div.i1ho8jml.dir.dir-ltr form.f114qjlg.dir.dir-ltr div#search-tabpanel.cbb5b2h.c1la6dqc.dir.dir-ltr div.ir2ixub.dir.dir-ltr div.c111bvlt.c1gh7ier.dir.dir-ltr div.c1ddhymz.cggll98.dir.dir-ltr div.s1t4vwjw.dir.dir-ltr button.bhtghtc.b1tqc7mb.dir.dir-ltr span.t1ng71ne.dir.dir-ltr span.t1dqvypu.dir.dir-ltr')
buscar$clickElement()

In [ ]:
pesquisar = rem_dr$findElement(using="css selector",'.c1nkokj4 > div:nth-child(1) > svg:nth-child(1) > path:nth-child(1)')
pesquisar$clickElement()

In [ ]:
DF = tibble::tibble(NOME = character(),URL = character(),PRECO = character())

qtd_paginas = rem_dr$findElement('xpath','/html/body/div[5]/div/div/div[1]/div/div[2]/div[2]/main/div[2]/div/div[3]/div/div/div/nav/div/a[4]')$getElementText()
qtd_paginas = as.integer(qtd_paginas) - 1

In [ ]:
for(i in 1:qtd_paginas){
  print(paste0("Página: ",i))
  
  quartos = rem_dr$findElements('xpath', '//*[@class="c1l1h97y dir dir-ltr"]')
  for(quarto in quartos){
    quarto$clickElement()
    janelas = rem_dr$getWindowHandles()
    rem_dr$switchToWindow(janelas[[2]])
    Sys.sleep(5)
    
    nome = rem_dr$findElement('xpath','//h1')
    nome = nome$getElementText()
    print(nome[[1]])
    
    url = rem_dr$getCurrentUrl()
    
    preco = rem_dr$findElement('css selector','._1qs94rc > span:nth-child(1) > span')
    preco = preco$getElementText()
    
    DF = DF |> tibble::add_row(NOME = nome[[1]],
                               URL = url[[1]],
                               PRECO = preco[[1]])
  
    rem_dr$closeWindow()
    rem_dr$switchToWindow(janelas[[1]])
    Sys.sleep(5)
  }
  proxima = rem_dr$findElement('xpath','//a[@aria-label="Próximo"]')
  proxima$clickElement()
  Sys.sleep(3)
}